<a href="https://colab.research.google.com/github/HanlunAI/ExhibitionDemonstration/blob/main/%E5%8F%AF%E4%BB%A5%E7%8E%A9%E5%91%B3%E7%9A%84%E8%88%AA%E9%81%93%E8%BB%8C%E8%B7%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

地球上的航道也是可以玩味的軌跡，我們先以球面的對稱性處理變分法問題(Calculus of variations) 。同學可以透過 Geogebra 互動體驗航道的設計。力所能及的同學更可以使用 Colab 進行更多探索，進而引申至一些後續問題供將來思考。

-- Harold Chao, YK Tai 2022

[互動素材：以一理想球體作為地球的模型，每座城市為球面上的一點，一航班需要通航於兩座城市之間，問如何尋得最短航道？](https://www.geogebra.org/classic/mesdgrmm)
*![picture](https://drive.google.com/uc?id=1qX8fh24E-I6CMspzWusgcvEwWzcVSrHR)*

[答：考慮過三點(城市A, 城市B, 地球中心)平面與球體相交的曲線，即可尋得最短航道。](https://www.geogebra.org/classic/wqwxrd6y)
*![picture](https://drive.google.com/uc?id=1TN-n61f6LWBRpGz4ugnzoCSbm2MQ63RQ)*

*![picture](https://drive.google.com/uc?id=121t0K1DF-VsH3rLKldfmVWGsJp0JSU31)*
[理解：因為球體的對稱性，以城市 A 為中心的圈圈，代表了由城市 A 出發可以到達指定距離的最遠邊界；同理，以城市 B 為中心也代表了由城市 A 出發可以到達指定距離的最遠邊界。最短航道上的點必然屬於以城市 A 的某圈與以城市 B 的某圈的相切點（否則，調整其中一圈收窄至於另一圈相切，便會產生更短航道），又由於對稱性，該點必處於過三點(城市A, 城市B, 地球中心)平面之上。於是我們可以考慮此平面與球體相交的曲線來尋找最短航道。](https://www.geogebra.org/classic/cjdrtfbb)

